In [ ]:
# test: python /data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/polypred.py --estimate-mixweights --betas /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/4_sbayesr/alt.sbayesr.snpRes,/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/alt.txt --pheno /data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/CAS/pheno/trait/data/alt_raw.txt --output-prefix /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/beta/alt --plink-exe /data1/jiapl_group/lishuhua/software/general/plink /data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/CAS/geno/CAS_for_PCA/CAS_qc_no_dups.bed

# python /data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/polypred.py --predict --betas /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/4_sbayesr/alt.sbayesr.snpRes,/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/alt.txt --mixweights-prefix /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/beta/alt --output-prefix /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/test_prs/alt --plink-exe /data1/jiapl_group/lishuhua/software/general/plink /data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/geno/Chinese/1_merged/merged.bed

In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import glob

def merge_all_jobs(gwas_path, jobs_dir, output_path, polyfun_exec):
    """
    Merge all job files in the specified directory into a single file.
    """
    cmd = f'python {polyfun_exec} --out-prefix {jobs_dir} --sumstats {gwas_path} --out {output_path} --adjust-beta-freq --allow-missing-jobs'
    os.system(cmd)

if __name__ == "__main__":
    gwas_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/1_parquet/"
    jobs_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/2_finemapping_jobs/"
    output_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/"
    polyfun_exec = "/data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/aggregate_finemapper_results.py"
    # trait_list = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi', 'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos', 'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'glucose', 'ggt', 'hdl', 'ldl', 'triglycerides', 'ua']
    trait_list = ['ua']

    for trait in trait_list:
        print(f"Processing trait: {trait}")
        output_path = os.path.join(output_base_dir, f"{trait}.txt")
        if os.path.exists(output_path):
            print(f"Output file {output_path} already exists. Skipping.")
            continue
        jobs_search_pattern = os.path.join(jobs_base_dir, trait, "job.chr*.gz")
        if not glob.glob(jobs_search_pattern):
            print(f"No job files found for trait {trait} in {jobs_base_dir}. Skipping.")
            continue
        jobs_num = len(glob.glob(jobs_search_pattern))
        # if jobs_num != 2750:
        #     print(f"Expected 2750 jobs for trait {trait}, but found {jobs_num}. Skipping.")
        #     continue
        gwas_path = os.path.join(gwas_base_dir, f"{trait}_snpvar.txt")
        jobs_dir = os.path.join(jobs_base_dir, trait, "job")
        merge_all_jobs(gwas_path, jobs_dir, output_path, polyfun_exec)

In [ ]:
# merge all beta files
import os
import pandas as pd
import numpy as np

def process_single_file(beta_file_path, pheno_file_path, output_file_path, polyfun_exec, plink_exec, bfile_path):
    cmd = f'python {polyfun_exec} --estimate-mixweights --betas {beta_file_path} --pheno {pheno_file_path} --output-prefix {output_file_path} --plink-exe {plink_exec} {bfile_path}'
    os.system(cmd)

if __name__ == "__main__":
    # trait_list = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi',
    #               'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos',
    #               'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'ggt',
    #               'glucose', 'hdl', 'ldl', 'triglycerides', 'ua']
    trait_list = ['ua']
    aggregate_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/"
    sbayesr_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/4_sbayesr/"
    pheno_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/CAS/pheno/trait/data/"
    beta_output_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/beta/"
    polyfun_exec = "/data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/polypred.py"
    plink_exec = "/data1/jiapl_group/lishuhua/software/general/plink"
    eas_bfile_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/CAS/geno/CAS_for_PCA/CAS_qc_no_dups.bed"

    for trait in trait_list:
        print(f"Processing trait: {trait}")
        aggregate_file_path = os.path.join(aggregate_base_dir, f"{trait}.txt")
        sbayesr_file_path = os.path.join(sbayesr_base_dir, f"{trait}.sbayesr.snpRes")
        betas_file_path = f'{sbayesr_file_path},{aggregate_file_path}'
        pheno_file_path = os.path.join(pheno_base_dir, f"{trait}_raw.txt")
        if not os.path.exists(aggregate_file_path) or not os.path.exists(sbayesr_file_path):
            print(f"Required files for trait {trait} do not exist. Skipping.")
            continue
        output_file_path = os.path.join(beta_output_base_dir, f"{trait}.txt.mixweights")
        output_prefix = os.path.join(beta_output_base_dir, f"{trait}.txt")
        if os.path.exists(output_file_path):
            print(f"Output file {output_file_path} already exists. Skipping.")
            continue
        process_single_file(betas_file_path, pheno_file_path, output_prefix, polyfun_exec, plink_exec, eas_bfile_path)
    

In [ ]:
# predict in test population
# python /data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/polypred.py --predict --betas /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/4_sbayesr/alt.sbayesr.snpRes,/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/alt.txt --mixweights-prefix /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/beta/alt --output-prefix /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/test_prs/alt --plink-exe /data1/jiapl_group/lishuhua/software/general/plink /data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/geno/Chinese/1_merged/merged.bed

import os
import pandas as pd
import numpy as np

def process_single_file(beta_file_path, mixweights_path, output_file_path, polyfun_exec, plink_exec, bfile_path):
    cmd = f'python {polyfun_exec} --predict --betas {beta_file_path} --mixweights-prefix {mixweights_path} --output-prefix {output_file_path} --plink-exe {plink_exec} {bfile_path}'
    os.system(cmd)

if __name__ == "__main__":
    trait_list = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi',
                  'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos',
                  'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'ggt',
                  'glucose', 'hdl', 'ldl', 'triglycerides', 'ua']
    aggregate_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/3_aggregate/"
    sbayesr_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/4_sbayesr/"
    mixweights_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/beta/"
    prs_output_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/res/test_prs/"
    polyfun_exec = "/data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/polypred.py"
    plink_exec = "/data1/jiapl_group/lishuhua/software/general/plink"
    eas_bfile_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/geno/Chinese/1_merged/merged.bed"

    for trait in trait_list:
        print(f"Processing trait: {trait}")
        aggregate_file_path = os.path.join(aggregate_base_dir, f"{trait}.txt")
        sbayesr_file_path = os.path.join(sbayesr_base_dir, f"{trait}.sbayesr.snpRes")
        betas_file_path = f'{sbayesr_file_path},{aggregate_file_path}'
        mixweights_prefix = os.path.join(mixweights_base_dir, f"{trait}.txt")
        if not os.path.exists(aggregate_file_path) or not os.path.exists(sbayesr_file_path) or not os.path.exists(f'{mixweights_prefix}.mixweights'):
            print(f"Required files for trait {trait} do not exist. Skipping.")
            continue
        output_file_path = os.path.join(prs_output_base_dir, f"{trait}")
        process_single_file(betas_file_path, mixweights_prefix, output_file_path, polyfun_exec, plink_exec, eas_bfile_path)
    